# Preprocessing and Feature Engineering

## Scaling

Many models, K-NN and linear models particularly, benefit substantially from scaling (notable exceptions are NN, and Tree based models). We cover a few methods readily available in Scikit-learn and other packages. (i denotes row, j column)  

`from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, Normalizer, MaxAbsScaler`  

**&#9755; StandardScaler:**  

$\large{x_{ij}' = \frac{x_{ij} - \mu_j}{\sigma_j}} \quad \forall i,j$

all scaled features now have zero mean and std of 1. Note that scaled data is not bounded since it is only scaled by the
standard deviation. An outlier before the transformation will still be an outlier after transformation. StandardScaler is sensitive to outliers as they skew the mean and standard deviation. 


**&#9755; MinMaxScaler**  

$\large{x_{i,j}' = \frac{x_{ij} - min(x_j)}{max(x_j) - min(x_j)}}$  

maps the minimum of each column to 0 and max to 1. It is easy to see in the transformation that MinMaxScaler is very sensitive to outliers  since it will cluster points in one area of the positive unit quadrant. Useful when data has clearly defined boundaries i.e. greyscale image.  


**&#9755; MaxAbsScaler**  
Similar to MinMaxScaler except min and max are measured in absolute value. Useful for sparse data. 

**&#9755; RobustScaler**  
Some robust statistics stuff (read interquartile range (IQR) and the median absolute deviation (MAD)). Similar to
StandardScaler, except robust to outliers.  

**&#9755; Normalizer**  
Projects unto the L1 or L0 unit ball (i.e. makes sure vectors have length 1 either in euclidean measure or L1 measure). Can't think of use cases atm.  

## Pitfalls

**&#9841; Scaling Sparse data:**  
Do not center sparse data (i.e. apply zero mean, unit variance or MinMaxScaler) since this will make the matrix not sparse anymore and blow up RAM and CPU. Scale by a constant factor since constant times zero is zero, preserving sparsity. Use MaxAbsScaler for sparse data.  

**&#9841; Including test set in scaler's `fit()`**  
Including the test set in `fit()` will lead to artificially higher accuracy scores since we are parameterizing our scaler based on our test set. In deployment, we obviously can't parameterize our scalers on unseen data. 

**&#9841; Calling `fit()` on training and test set separately:**  
Never call `fit()` on the test set separately since it will change the relationship of test points to training points. Only call fit on training set. Then call transform on both training and test sets. 

**&#9841; Not using pipelines in cross validation:**  
When you perform cross validation on the scaled training set, the validation fold is scaled in the same way as the training fold, leading to pitfall 2. Using pipelines solves this pitfall.

<img src="files/images/scaling.png">

## Detour: Pipelines

In [15]:
from sklearn.pipeline import make_pipeline

# this is wrapper for Pipeline below
pipe = make_pipeline(StandardScaler(), Ridge())
pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)

# this gives flexibility in naming the steps. Useful when tuning parameters.
from sklearn.pipeline import Pipeline
pipe = Pipeline((("scaler", StandardScaler()),
                 ("regressor", KNeighborsRegressor)))

# cross validation with pipeline
knn_pipe = make_pipeline(StandardScaler(), KNeighborsRegressor())
scores = cross_val_score(knn_pipe, X_train, y_train, cv=10)
np.mean(scores), np.std(scores)


# pipeline and GridSearchCV. Use modified names ([step_name]__[parameter]) 
# in GridSearch param_grid
knn_pipe = make_pipeline(('scale', StandardScaler()), ('model', KNeighborsRegressor())
param_grid = {'model__n_neighbors': range(1, 10)}
grid = GridSearchCV(knn_pipe, param_grid, cv=10)
grid.fit(X_train, y_train)
print(grid.best_params_)
print(grid.score(X_test, y_test))

## Feature Distributions 

Linear Models may perform better when features are normally distributed. Scaling data does not change the distribution of the points. There are several transformations available that do just this. Most common tranformations are power transformations, particularly the Box Cox Transformation.  

**&#9755; Box-Cox transformation:**  

$bc_{\lambda}(x) = \cases{\frac{x^\lambda - 1}{\lambda} & \text{if } \lambda \neq 0\cr log(x) & \text{if } \lambda = 0 }$  

only applicable for positive x!  



In [ ]:
# sklearn 0.20-dev
from sklearn.preprocessing import PowerTransformer
pt = PowerTransformer(method='box-cox') 
# soon: method='Yeo-Johnson'
pt.fit(X)

## Categorical Data  

We can one-hot encode a feature's categorical values (e.g. $\{'green', 'blue', 'yellow'\}$) using pandas. We can encode all k values, introducing redundancies, or k-1 to have more accurate interpretation of coefficients. We can also one hot encode using sklearn but this method requires categorical values to be in integer format.  
  
We face a problem when categorical feature values are of high cardiniality (50, 100, etc). We must find a way to compress the values into fewer than k features. This solution is often specific to the dataset. 

## Pitfalls

**&#9841; Categorical Values in test but not training set:**  
Make sure to one hot encode all possible values of the feature, not just those in the test and traning sets.

In [ ]:
import pandas as pd
df = pd.DataFrame({'salary': [103, 89, 142, 54, 63, 219],
                   'boro': ['Manhattan', 'Queens', 'Manhattan',
                            'Brooklyn', 'Brooklyn', 'Bronx']})

# note that there are more categories in the definition of the 
# column than those seen in the column
df['boro'] = pd.Categorical(
    df.boro, categories=['Manhattan', 'Queens', 'Brooklyn', 'Bronx', 'Staten Island'])
pd.get_dummies(df, columns=['boro'])

## Feature interactions  

Linear models particularly benefit from feature interactions. Feature transformations allow models to fit non linear boundaries or curves to the data. However, these transformations blow up the feature space. Using kernel transformations allow for the power of the transformations without a significant increase in CPU or memory. 

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

# Imputation  

Imputation refers to dealing with missing values. We can always drop columns with NaN values but we would lose all the information given by non NaN values. We could also drop the observations that contain NaN values if they are few. There are 4 types of imputation methods that try to extrapolate missing values: Mean/median, kNN, regression, probabilistic. A general good practice with imputation is to create a dummy indicating if the value was NaN in addition to extrapolation of the value. Check out fancyimpute library (note that fancy impute does not implement fit/transform paradigm, thus does not work with pipelines...information leak!)

**&#9755; Mean Imputation:**  
Mean imputation fills in NaN values with the mean of the given feature. This, of course, only works for non binary data. Mean imputation can be acceptable if missing values are few. Else, this method can destroy the data distribution, hiding useful structural relationships between the input and target data.  

**&#9755; kNN Imputation:**  
kNN imputation works by taking k nearest neighbors of the observation with the missing value, and replaces NaN with the average value of that feature among the k neighbors. Note that kNN imputation only works if the features used to compute distance are not NaN, since Euclidiean distance is undefined for NaN values. So points with NaN values are thown away when computing  k neighbors.  

**&#9755; Model Driven Imputation:**  
Train a model on the non missing features to predict missing features (kNN imputation is arguably Model driven imputation but there are some slight differences). A popular model to use is random forests (see code below). 

**&#9755; (MICE) Multiple Imputation by Chained Equations:**  
Not sure yet how it works but this is a very popular imputation method. Not in sklearn, but is in fancyimpute. 

In [19]:
# mean imputation
from sklean.preprocessing import Imputer

imp = Imputer(strategy='mean').fit(X_train)
imp.transform(X_train)


# kNN imputation: very inefficient didactic implementation
# use library!
distances = np.zeros((X_train.shape[0], X_train.shape[0]))
for i, x1 in enumerate(X_train):
    for j, x2 in enumerate(X_train):
        dist = (x1 - x2) ** 2
        nan_mask = np.isnan(dist)
        distances[i, j] = dist[~nan_mask].mean() * X_train.shape[1]
neighbors = np.argsort(distances, axis=1)[:, 1:]
n_neighbors = 3
X_train_knn = X_train.copy()
for feature in range(X_train.shape[1]):
    has_missing_value = np.isnan(X_train[:, feature])
    for row in np.where(has_missing_value)[0]:
        neighbor_features = X_train[neighbors[row], feature]
        non_nan_neighbors = neighbor_features[~np.isnan(neighbor_features)]
        X_train_knn[row, feature] = non_nan_neighbors[:n_neighbors].mean()
    
    
# Model Driven Imputation with random forests.     
rf = RandomForestRegressor(n_estimators=100)
X_imputed = X_train.copy()
for i in range(10):
    last = X_imputed.copy()
    for feature in range(X_train.shape[1]):
        inds_not_f = np.arange(X_train.shape[1])
        inds_not_f = inds_not_f[inds_not_f != feature]
        f_missing = np.isnan(X_train[:, feature])
        rf.fit(X_imputed[~f_missing][:, inds_not_f], X_train[~f_missing, feature])
        X_imputed[f_missing, feature] = rf.predict(
            X_imputed[f_missing][:, inds_not_f])
    if (np.linalg.norm(last - X_imputed)) < .5:
        break
scores = cross_val_score(logreg, X_imputed, y_train, cv=10)
np.mean(scores)


# MICE with fancyimpute
import fancyimpute

mice = fancyimpute.MICE(verbose=0)
X_train_fancy_mice = mice.complete(X_train)
scores = cross_val_score(logreg, X_train_fancy_mice, y_train, cv=10)
scores.mean()

SyntaxError: invalid syntax (<ipython-input-19-dc41b017f0c6>, line 44)

<img src="files/images/imputation_methods.png">

## Feature Selection  

It is often beneficial to reduce the feature space. It leads to faster predictions and faster traning times. Less data to gather when model is in production. Less storage for model and dataset and more importantly, leads to more interpretable models. There are supervised and unsupervised feature selection methods.  

**&#9755; Covariance based:**  
When two features are highly correlated, we can remove 1 of them without substantially affecting model predictions (probably..it could be the small difference in variation of these features was really important for predictions. We can test this easily though.) Show code for sorting correlation matrix heatmap.

**&#9755; PCA:**  
PCA maps the data to a linear subpsace. Only reduces feature space for training and predictions, not for data gathering. It makes model less interpretable BUT it can lead to very useful visualizations in 2D and 3D and it can speed up training time considerably. It can remove useful information. 

**&#9755; Univariate Statistics:**  
We can build a linear regression model on single feature and the target and measure F and p values for the coefficient. These values let us know if feature is important for prediction. However, linear regression assumes linear relationship between input and target, which may be a poor assumption. Furthermore, the linear assumption decreases the importance of binary features.  

Mutual information relies on nonparametric methods based on entropy estimation from k-nearest neighbors distances. MI Can deal with binary features (must specify which columns are binary). This is a good choice when assuming non linear interaction between feature and target. However, much more computationally intensive than f regression.  

**&#9755; Model Based feature selection:**  
Train a model and check coefficients (in the case of linear models) or splits (in the case of tree based models). Discard features with coefficient close to zero or without/non informative splits. This is usually a good choice but computationally expensive. 

**&#9755; Iterative Model based feature selection:**  
Removing many features at once may change the model more than we indend to. A better way to iteratively remove a single feature, each time retraining the model and selecting the least important. We can do this with RFE (Recursive Feature Elimination) in Sklearn. We can also iterate the opposite way, Sequential Feature Selection from library mlextend. 


## Pitfalls  

**&#9841; Univariate statistics do not account for correlation:**  
Univariate statistics give highly correlated features the same importance. Model driven feature selection accounts for correlation and can thus give much higher importance to some features while removing importance from others (which is which is usually random).

In [ ]:
# Get F and p values so we can visualize and drop features
from sklearn.feature_selection import f_regression

f_values, p_values = f_regression(X, y)


# or we can use SelectKBest to automatically drop features below a threshold. 
from sklearn.feature_selection import SelectKBest, SelectPercentile, SelectFpr
from sklearn.linear_model import RidgeCV


select = SelectKBest(k=2, score_func=f_regression)
select.fit(X_train, y_train)
print(X_train.shape)
print(select.transform(X_train).shape)

# SelectKBest with pipelines
make_pipeline(StandardScaler(), SelectKBest(k=2, score_func=f_regression), RidgeCV())

# Mutual information
from sklearn.feature_selection import mutual_info_regression

scores = mutual_info_regression(X_train, y_train, discrete_features=[3])

# Model Driven feature selection
from sklearn.feature_selection import SelectFromModel

select_lassocv = SelectFromModel(LassoCV(), threshold=1e-5)
select_lassocv.fit(X_train, y_train)
print(select_lassocv.transform(X_train).shape)


# Recursive Feature Elimination
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE

# create ranking among all features by selecting only one
rfe = RFE(LinearRegression(), n_features_to_select=1)
rfe.fit(X_train_scaled, y_train)
rfe.ranking_

# RFE that automatically selects best number of features to keep
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFECV

rfe = RFECV(LinearRegression(), cv=10)
rfe.fit(X_train_scaled, y_train)
print(rfe.support_)
print(boston.feature_names[rfe.support_])

# Sequential Feature Selection
from mlxtend.feature_selection import SequentialFeatureSelector

sfs = SequentialFeatureSelector(LinearRegression(), forward=False, k_features=7)
sfs.fit(X_train_scaled, y_train)


# Dimensionality Reduction  

Transforming data using unsupervised learning can have many motivations. The most common motivations are visualization, compressing the data, and finding a representation that is more informative for further processing.

## PCA

On a high level, principal component analysis iteratively does the following: 

Finds the direction, orthogonal to all previously established directions, along which the data points vary the most (highest variance). Describe this direction with a unit vector. Continue until we have n directions. Build an n-dimensional linear map with these n vectors.

These n vectors, representing n directions are called principal components. For n dimensional data, an n-dimensional PCA is just a rotation (re-basing of the vector space) in which the dimensions are sorted in order of decreasing variance. 

We usually scale the the data to have unit standard deviationg before applying PCA (PCA normalizes to 0 mean under the covers. Remember huge improvements in PCA benefits after applying StandardScaler on data in hw4 problem). It is important to note that PCA is an unsupervised method, and does not use any class information when finding the direction of maxiumum variance. It simply looks at the correlations in the data. PCA can substantially reduce the input data's dimensionality, greatly speeding up model training. PCA can also lead to increased model accuracy since dimensionality reduction decreased overfitting, which can in turn result in a better model. 

TODO talk about PCA equation  

TODO Whitening

TODO kernel PCA

## Pitfalls  

**&#9841; Uneven class distributions**  
When our dataset has skewed distributions (or outliers), PCA will give a lot of weight to the classes with many observations. We need to randomly throw out observations of commonly occuring classes so most classes have similar occurence


<img src="files/images/pca-intuition.png">

In [ ]:
from sklearn.decomposition import PCA
X_train_scaled = StandardScaler(X_train)
pca = PCA(n_components=2)
X_train_pca = pca.fit(X_train_scaled)

## Manifold learninf: t-SNE

While PCA is often a good first approach for transforming your data so that you might be able to visualize it, the nature of the method (applying a rotation and then dropping directions) limits its usefulness. There is a class of algorithms for visualization called manifold learning algorithms that allow for much more complex mappings and often provide better visualizations. A particularly useful one is the t-SNE algorithm. A manifold is a topological space that locally resembles Euclidean space near each point. More precisely, each point of an n-dimensional manifold has a neighbourhood that is homeomorphic to the Euclidean space of dimension n. Manifold learning algorithms are mainly aimed at visualization, and so are rarely
used to generate more than two new features.

### t-distributed stochastic neighbor embedding

The idea behind t-SNE is to find a two-dimensional representation of the data that preserves
the distances between points as best as possible. t-SNE starts with a random twodimensional
representation for each data point, and then tries to make points that are close in the original feature space closer, and points that are far apart in the original feature space farther apart.


`from sklearn.manifold import TSNE
tsne = TSNE(random_state=42)
digits_tsne = tsne.fit_transform(digits.data)`


t-SNE transform does not work well for prediction. It is solely useful as a visualization method (maybe a future modification to the algorithm lets t-SNE transform allow for new points). A general rule of thumb is that larger perplexity works better for large datasets and lower perplexity works well for smaller data sets.

<img src="files/images/pca-digits.png">

<img src="files/images/tsne-digits.png">

## Linear Discriminant Anlysis

Discriminant Analysis is actually a classification method but it has succesfully been used as a dimensionality reduction method. It is quite simple due to its generative probabilistic assumptions.

$\large{P(y=k | X) = \frac{P(X | y=k) P(y=k)}{P(X)} = \frac{P(X | y=k) P(y = k)}{ \sum_{l} P(X | y=l) \cdot P(y=l)}}$

$\large{p(X | y=k) = \frac{1}{(2\pi)^n |\Sigma|^{1/2}}\exp\left(-\frac{1}{2} (X-\mu_k)^t \Sigma^{-1} (X-\mu_k)\right)}$

From the first equation above, we can see that it is just Bayes rule combined with an assumption regarding the underlying distribution of the data. In Linear Discriminant Analysis, we assume a gaussian distribution. Note the covariance matrix $\Sigma^{-1}$ in the second equation. 


### PCA vs Linear Discriminant Analysis

- LDA is a supervised alternative to PCA.
- Both fit Gaussian model
- PCA for the whole data
- LDA multiple Gaussians with shared covariance
- Can use LDA to transform space!
- At most as many components as there are classes (needs between class variance)




## Unsupervised Learning

Unsupervised learning is more often used for exploratory analysis and visualization rather than directly in a classifier. 

### Clustering
**&#9755; k-Means Clustering:** Simple, iterative algorithm: pick k random points as cluster centers. While cluster centers change (more than some epsilon), assign each data point to its closest cluster center, then recompute cluster centers as the mean of the assigned points. We often don't know k. How do we go about choosing it? Pick a few different k's at random. k-Means finds a *local* minimum of the following cost function.

$$ \min_{\mathbf{c}_j \in \mathbf{R}^p, j=1,..,k} \sum_i ||\mathbf{x}_i - \mathbf{c}_{x_i}||^2$$

k-Means can also be used for feature extraction. We can add a categorical feature denoting cluster membership and/or a continuous feature denoting distance from cluster.(Note, k-Means default in scikit learn runs k-means 10 times for different initializations and returns best one)

- scaling greatly affects k-means result
- Since k-Means returns a local minimum, it is very sensitive to initialization. 
- Cluster areas are convex sets, limiting the complexity of the boundaries. 
- Does not address cluster sizes. 


**&#9755; Agglomerative Clustering:** refers to a collection of clustering algorithms that build upon the same principles: the algorithm starts by declaring each point its own cluster,
and then greedily merges the two most similar clusters until some stopping criterion is satisfied. Three merging (linkage) criterions are implemented in scikit-learn: ward, average, and complete. Ward returns relatively equally sized clusters. We can use scipy to plot dendograms to visualize the development of the agglomerative clustering algorithm. Agglomerative clustering cannot transform new points.


**&#9755; DBSCAN:** Density based spatial clustering of applications with noise. DBSCAN does not need the number of clusters to be set a piori, which can be advantageous in some applications. This method however, is very sensitive to its two parameters (min_samples, epsilon). DBSCAN often results in highly imbalanced cluster sizes. DBSCAN IS TRASH IF YOU DON'T HAVE CLASS LABELS.

#### K-Means Clustering

In [ ]:
from sklean.cluster import KMeans
km = KMeans(n_clusters=5, random_state=0)
km.fit(X)
km.transform(X_test)
plt.scatter(X[:, 0], X[:, 1], c=km.labels_)  #2d data
plt.gca().set_aspect("equal")

#### Agglomerative Clustering

In [ ]:
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram, linkage

agg = AgglomerativeClustering(n_clusters=3)
assignment = agg.fit_predict(X)
ax = plt.subplot()
ax.scatter(X[:, 0], X[:, 1], assignment) #2d data

# dendogram visualization 
linkage_array = linkage(X, 'ward')
dendrogram(linkage_array, p=30, truncate_mode='level')
plt.show()

#### DBSCAN

In [ ]:
from sklearn.cluster import DBSCAN

dbscan = DBSCAN()
clusters = dbscan.fit_predict(X)

### Cluster Evaluation

**&#9755; NMI:** Normalized Mutual Information uses the information theory metric of mutual information to measure clustering results to grounds truths. 

**&#9755; ARI:** Adjusted Rand Index evaluates clusterings by comparing clusters to groundd truths.   

**&#9755; Silhouette Score:** An unsupervised cluster evaluation score that favors compact cluster and therefore favors methods like k-Means over DBSCAN. 

**&#9755; Silhouette Score:**

# NMF

Non-negative Matrix Factorization. 

# Outlier Detection


Useful for fraud detection, network failure, intrusion detection in networks etc. 

Outlier Detection: data set is dirty, data contains outliers

Novelty detection: data will be clean. You will be given a new data set and asked to find new points. 

in both you want to identify 'different' data.

Idea, model data distribution p(X). If point is unlikely under model, then it is classified as an outlier. x

# Choosing data representation

This is among the harder tasks in machine learning. There are a few methods used to verify our representation of the data is 'good'. The simplest verification method is measuring the relative improvement between the data representations using 1-NN classifier. Since 1-NN only relies on the representation of the data, if some 1-NN performs better on one representation, then it indicates our representation of data is better. 